In [1]:
from win32api import GetMonitorInfo, MonitorFromPoint

monitor_info = GetMonitorInfo(MonitorFromPoint((0,0)))
monitor_area = monitor_info.get("Monitor")
work_area = monitor_info.get("Work")

taskbar_height = monitor_area[3] - work_area[3]
print("Chiều cao của thanh taskbar là {}.".format(taskbar_height))

Chiều cao của thanh taskbar là 48.


In [2]:
from faker import Faker
from random import randint

# Create a Faker instance for Vietnam and another one for the US
fake_vn = Faker('vi_VN')
fake_us = Faker('en_US')

# Generate fake data
data_set = []
for _ in range(30):  # 15 Vietnamese names
    data = {
        "fullname": fake_vn.name(),
        "dob": fake_vn.date_of_birth(minimum_age=18, maximum_age=70).isoformat(),
        "sex": fake_vn.random_element(elements=("Male", "Female")),
        "height": randint(150, 200),  # Random height in cm
        "weight": randint(50, 100),  # Random weight in kg
        "phone": fake_vn.phone_number(),
        "insur_number": fake_vn.random_number(digits=6, fix_len=True),
        "address": fake_vn.address().replace("\n", ", "),
        "note": ""  # Note is empty
    }
    data_set.append(data)

# for _ in range(30):  # 15 US names
#     data = {
#         "fullname": fake_us.name(),
#         "dob": fake_us.date_of_birth(minimum_age=18, maximum_age=70).isoformat(),
#         "sex": fake_us.random_element(elements=("Male", "Female")),
#         "height": randint(160, 200),  # Random height in cm
#         "weight": randint(60, 110),  # Random weight in kg
#         "phone": fake_us.phone_number(),
#         "insur_number": fake_us.random_number(digits=6, fix_len=True),
#         "address": fake_us.address().replace("\n", ", "),
#         "note": ""  # Note is empty
#     }
#     data_set.append(data)

# Print the data set
for data in data_set:
    print(data)


{'fullname': 'James Pugh', 'dob': '1958-09-07', 'sex': 'Male', 'height': 191, 'weight': 51, 'phone': '653.474.7599x3531', 'insur_number': 899704, 'address': '0842 Houston Lane, New Andrea, MH 57620', 'note': ''}
{'fullname': 'Tyler Gibson', 'dob': '1984-10-02', 'sex': 'Female', 'height': 166, 'weight': 54, 'phone': '+1-821-954-0779', 'insur_number': 186059, 'address': 'USNS Eaton, FPO AA 73743', 'note': ''}
{'fullname': 'Tyrone Fitzpatrick', 'dob': '1986-05-03', 'sex': 'Female', 'height': 161, 'weight': 66, 'phone': '(473)691-4242', 'insur_number': 196112, 'address': '35292 Freeman Vista, Janetstad, MI 09202', 'note': ''}
{'fullname': 'Tony Clark', 'dob': '1998-07-29', 'sex': 'Male', 'height': 180, 'weight': 84, 'phone': '(929)696-8407x145', 'insur_number': 829424, 'address': 'Unit 7786 Box 4952, DPO AA 69374', 'note': ''}
{'fullname': 'Michael Watson', 'dob': '1994-07-07', 'sex': 'Male', 'height': 152, 'weight': 94, 'phone': '(848)293-6175x80858', 'insur_number': 586156, 'address': '3

In [3]:
from support_function import *
print(get_tables('Pham Huynh Quoc Thien'))

['data_ecg_phamhuynhquocthien_20231119_232824', 'data_ecg_phamhuynhquocthien_20231124_212507', 'data_ecg_phamhuynhquocthien_20231124_220725', 'data_ecg_phamhuynhquocthien_20231124_221226', 'data_ecg_phamhuynhquocthien_20231124_225438']


In [10]:
import sys
import os
import pandas as pd
import psycopg2 as ps
from datetime import datetime
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import QSizePolicy,QVBoxLayout
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
#Function connect to database
def connect_to_db():
    connectDB = ps.connect(
        host="a-3.postgres.database.azure.com",
        dbname="project",
        user="Project_Health_Care_System",
        password="@healthcare2",
        port=5432
    )
    return connectDB.cursor()

def get_data_from_table(table_name):
    cursor = connect_to_db()
    cursor.execute(f"SELECT X, Y FROM {table_name}")
    data = cursor.fetchall()
    x_values = [row[0] for row in data]
    y_values = [row[1] for row in data]
    return x_values, y_values

import pyqtgraph as pg
from PyQt5 import QtWidgets, QtCore

class MainWindow(QtWidgets.QMainWindow):
    def __init__(self, parent=None):
        super(MainWindow, self).__init__(parent)
        self.graphWidget = pg.PlotWidget()
        self.setCentralWidget(self.graphWidget)

        self.x, self.y = get_data_from_table('data_ecg_phamhuynhquocthien_20231124_225438')
        self.data_line = self.graphWidget.plot(self.x[:500], self.y[:500], pen=pg.mkPen('r', width=2))

        self.current_index = 0
        self.graphWidget.setXRange(self.x[0], self.x[0] + 500)
        self.graphWidget.setBackground('w')

        self.timer = QtCore.QTimer()
        self.timer.setInterval(2)  # in milliseconds
        self.timer.timeout.connect(self.update_plot)
        self.timer.start()

    def update_plot(self):
        self.current_index += 1
        if self.current_index > len(self.x) - 500:
            self.timer.stop()  # Dừng QTimer khi đến điểm cuối của dữ liệu
            return
        self.data_line.setData(self.x[:self.current_index+500], self.y[:self.current_index+500])
        self.graphWidget.setXRange(self.x[self.current_index], self.x[self.current_index] + 500)

def main():
    app = QtWidgets.QApplication([])
    main = MainWindow()
    main.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
